![](./图片1.png)

# 第一部分

**迁移学习**：把在 source domain 任务上的学习到的模型应用到 target domain 的任务。

Finetune 就是一种迁移学习的方法。比如做人脸识别，可以把 ImageNet 看作 source domain，人脸数据集看作 target domain。通常来说 source domain 要比 target domain 大得多。可以利用 ImageNet 训练好的网络应用到人脸识别中。

对于一个模型，通常可以分为前面的 feature extractor（卷积层）和后面的 classifier（全连接层），在 Finetune 时，通常不改变 feature extractor（卷积层）的权值，也就是冻结卷积层；并且改变最后一个全连接层的输出来适应目标任务（不同的 classes），训练后面 classifier 的权值，这就是 Finetune。通常 target domain 的数据比较小，不足以训练全部参数，容易导致过拟合，因此不改变 feature extractor 的权值。

<br/>

# 第二部分

**Finetune 步骤如下**：
- 获取预训练好模型的参数
- 使用 load_state_dict() 把参数加载到模型中
- 修改替换全连接层/输出层
- 冻结 feature extractor（卷积层）的参数

**冻结 feature extractor 的参数通常有 2 种做法**：
- 方法一：设置 requires_grad=False 或者 lr=0
- 方法二：在优化器中通过 params_group 给 feature extractor（卷积层）设置一个较小的学习率

**代码**：
```
# ======================================= 在设置模型环节的操作 ================================================
import torchvision

# 提前创建一个模型对象，以 resnet18 为例
resnet18_ft = torchvision.models.resnet18()

# 获取预训练好模型的参数
# 下载地址：http://download.pytorch.org/models/resnet18-5c106cde.pth

# 使用 load_state_dict() 把参数加载到模型中
state_dict_load = torch.load(path_pretrained_model)
resnet18_ft.load_state_dict(state_dict_load)

# 冻结 feature extractor（卷积层）的参数，方法一：
# 先冻结所有参数，然后再修改替换全连接层，相当于冻结了卷积层的参数
for param in resnet18_ft.parameters():  # 冻结所有参数
    param.requires_grad = False

num_ftrs = resnet18_ft.fc.in_features  # 拿到 fc 层的输入个数，num_ftrs 是由模型 fc 之前的网络层决定的每个样本对应的参数数量
resnet18_ft.fc = nn.Linear(num_ftrs, classes)  # 构造新的 fc 层替换原来的 fc 层，classes 为分类类别数量，是二分类还是几分类


# ======================================= 在设置优化器环节的操作 ================================================
# 冻结 feature extractor（卷积层）的参数，方法二：
# 在优化器中通过 params_group 给 feature extractor（卷积层）设置一个较小的学习率
fc_params_id = list(map(id, resnet18_ft.fc.parameters()))  # # 获取全连接层参数的内存地址
base_params = filter(lambda p: id(p) not in fc_params_id, resnet18_ft.parameters())  # 使用 filter 过滤不属于全连接层的参数，也就是保留卷积层参数的内存地址
optimizer = optim.SGD([{'params': base_params, 'lr': LR*0.1}, {'params': resnet18_ft.fc.parameters(), 'lr': LR}], momentum=0.9)  # 设置优化器的分组学习率，传入一个 list，包含 2 个元素，每个元素是字典，对应 2 个参数组
```

**方法一的效率比较高，节约时间；方法二比较灵活，可彻底冻结卷积层的参数，也可不彻底冻结，把卷积层的学习率设置为 0，即可实现方法一的效果。**

<br/>

# 第三部分

**PyTorch 模型使用 GPU，可以分为 3 步**：
- 首先设置 device：`device = torch.device("cuda" if torch.cuda.is_available() else "cpu")`
- 把模型加载到 device：`resnet18_ft.to(device)`
- 在训练环节从 train_loader 中读取数据的循环中，把每个 batch 的数据和 label 加载到 device：
```
for i, data in enumerate(train_loader):
    # forward
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = resnet18_ft(inputs)
```